In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import *
import os

from random import *

sns.set(style='white', context='notebook', palette='deep')

In [ ]:
# define train set
from google.colab import drive
drive.mount('/content/drive')
train = pd.read_csv('./drive/My Drive/DACON/data_file/train.csv')
test = pd.read_csv('./drive/My Drive/DACON/data_file/test.csv')
train_copy = train.copy()
test_copy = test.copy()

In [ ]:
rot_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=45, # rotation range 1이 최대로 움직인 각도 : 45도
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0,    
    zoom_range=0,     
    channel_shift_range=0.0,
    fill_mode='constant', 
    cval=0.0,             
    horizontal_flip=False,
    vertical_flip=False,   
    rescale=1./255, 
    preprocessing_function=None,
    data_format=None,
    validation_split=0,
    dtype=None
)

trans_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, 
    width_shift_range=0.2, 
    height_shift_range=0.2,
    brightness_range=None,
    shear_range=0,     
    zoom_range=0,      
    channel_shift_range=0.0,
    fill_mode='constant',
    cval=0.0,             
    horizontal_flip=False, 
    vertical_flip=False,   
    rescale=1./255, 
    preprocessing_function=None,
    data_format=None,
    validation_split=0, 
    dtype=None
)

shear_zoom_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, 
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0.2,    
    zoom_range=0.2,      
    channel_shift_range=0.0,
    fill_mode='constant', 
    cval=0.0,             
    horizontal_flip=False, 
    vertical_flip=False,   
    rescale=1./255, 
    preprocessing_function=None,
    data_format=None,
    validation_split=0, 
    dtype=None
)

flip_gen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=0, 
    width_shift_range=0.0,
    height_shift_range=0.0,
    brightness_range=None,
    shear_range=0,     
    zoom_range=0,      
    channel_shift_range=0.0,
    fill_mode='constant', 
    cval=0.0,             
    horizontal_flip=True, 
    vertical_flip=True,   
    rescale=1./255, 
    preprocessing_function=None,
    data_format=None,
    validation_split=0, 
    dtype=None
)

In [ ]:
def augmentation( input_imgs, aug_size ):
    df = input_imgs
    new_data_set = []
    num_of_training_set = df.shape[0]

    for i in range(num_of_training_set//2):
        rand_1 = np.random.randint(num_of_training_set)
        rand_2 = np.random.randint(num_of_training_set)
        rand_3 = np.random.randint(num_of_training_set)
        rand_4 = np.random.randint(num_of_training_set)
    
        for j in range(aug_size):
            # rotation
            _rot = rot_gen.flow( np.array(df.iloc[rand_1,3:]).reshape(1,28,28,1) ).next().reshape(784,)
            new_data_set += [[
                df.iloc[rand_1,1],
                df.iloc[rand_1,2],
            ] + list(_rot)]
            # translation
            _trans = trans_gen.flow( np.array(df.iloc[rand_2,3:]).reshape(1,28,28,1) ).next().reshape(784,)
            new_data_set += [[
                df.iloc[rand_2,1],
                df.iloc[rand_2,2],
            ] + list(_trans)]
            # shear / zoom
            _shear = shear_zoom_gen.flow( np.array(df.iloc[rand_3,3:]).reshape(1,28,28,1) ).next().reshape(784,)
            new_data_set += [[
                df.iloc[rand_3,1],
                df.iloc[rand_3,2],
            ] + list(_shear)]
            # flip
            _flip = flip_gen.flow( np.array(df.iloc[rand_4,3:]).reshape(1,28,28,1) ).next().reshape(784,)
            new_data_set += [[
                df.iloc[rand_4,1],
                df.iloc[rand_4,2],
            ] + list(_flip)]

    columns = ['digit', 'letter'] + [str(x) for x in range(784)]
    aug = pd.DataFrame(new_data_set, columns=columns)

    train_norm = pd.concat([ input_imgs.iloc[:,1:3], np.divide(input_imgs.iloc[:,3:],255) ],axis=1)
    train_aug = pd.concat([train_norm,aug])

    return train_aug


In [ ]:
def train_test_gen(input_imgs, aug_size):
    train_aug = augmentation(input_imgs, aug_size)

    x_train = train_aug.iloc[:,2:].values.copy()
    x_train = x_train.reshape(-1,28,28,1)

    y_train = train_aug['digit']
    y_train = to_categorical(y_train,num_classes = 10)

    return train_test_split(x_train,y_train,test_size=0.1,random_state=randint(1,100))

In [ ]:
def load_best(file_name):
    filepath = './drive/My Drive/DACON/saved_model/' + file_name + '/'
    time_list = []
    for f_name in os.listdir(f"{filepath}"):
        written_time = os.path.getctime(f"{filepath}{f_name}")
        time_list.append((f_name, written_time))
    sorted_file_list = sorted(time_list, key=lambda x: x[1], reverse=True)
    best = sorted_file_list[0]
    best_name = best[0]
    model = load_model( filepath + best_name )
    print('\033[31m' + best_name + '\033[0m')
    print()
    return model

In [ ]:
def ensemble(input_imgs):
    pred = []
    L = input_imgs.shape[0]
    label_list = np.zeros((L,10))
    for i in range(num):
        label = model_list[i].predict( input_imgs )
        label_list += label
        # label_list += label*acc_list[i]
    for j in range(len(label_list)):
        pred.append( np.argmax(label_list[j]) )

    return pred

In [ ]:
def compare(file1,file2):
    filepath1 = './drive/My Drive/DACON/submission/' + file1 +'.csv'
    filepath2 = './drive/My Drive/DACON/submission/' + file2 +'.csv'
    f1 = pd.read_csv(filepath1)
    f2 = pd.read_csv(filepath2)
    match = np.array( [ f1['digit']==f2['digit'] ][0] )
    acc = len( np.where(match==True)[0] )/len(match)
    return acc

In [ ]:
def pred_acc(file_name,file_list):
    score = []
    for i in range( len(file_list) ):
        acc = compare(file_name, file_list[i])
        score.append(acc)
        print( 'Compared with ' + file_list[i].replace('submision_','') + ' : {}'.format(acc) )
    #return score
    

In [ ]:
def get_model(learning_rate):
    
    # Remove the previous model.
    model = None
    
    # Input layer
    img_input = Input(shape = (28,28,1))
    
    # CNN
    # Identity mapping shortcut을 위한 conv_1 layer
    conv_1 = Conv2D(128, kernel_size = 3, padding = 'same', activation = 'relu')(img_input)
    conv_1_bn = BatchNormalization()(conv_1) 
    
    conv_2_1 = Conv2D(128, kernel_size = 3, padding = 'same', activation = 'relu')(conv_1_bn)
    conv_2_1 = Conv2D(128, kernel_size = 3, padding = 'same')(conv_2_1)
    conv_2_1_bn = BatchNormalization()(conv_2_1)
    
    # ShortCut connection
    add_2_1 = add([conv_1_bn, conv_2_1_bn])
    out_2_1 = Activation('relu')(add_2_1)
    out_2_1_bn = BatchNormalization()(out_2_1)
    
    conv_2_2 = Conv2D(128, kernel_size = 3, padding = 'same', activation = 'relu')(out_2_1_bn)
    conv_2_2 = Conv2D(128, kernel_size = 3, padding = 'same')(conv_2_2)
    conv_2_2_bn = BatchNormalization()(conv_2_2)
    
    # ShortCut connection
    add_2_2 = add([out_2_1_bn, conv_2_2_bn])
    out_2_2 = Activation('relu')(add_2_1)
    out_2_2_bn = BatchNormalization()(out_2_2)
    
    pool_2 = MaxPool2D((2,2), strides = 2)(out_2_2_bn)
    
    conv_3_0 = Conv2D(256, kernel_size = 1, strides = 1)(pool_2)
    conv_3_0_bn = BatchNormalization()(conv_3_0)
    
    conv_3_1 = Conv2D(256, kernel_size = 3, padding = 'same', activation = 'relu')(conv_3_0_bn)
    conv_3_1 = Conv2D(256, kernel_size = 3, padding = 'same')(conv_3_1)
    conv_3_1_bn = BatchNormalization()(conv_3_1)
    
    # ShortCut connection
    add_3_1 = add([conv_3_0_bn, conv_3_1_bn])
    out_3_1 = Activation('relu')(add_3_1)
    out_3_1_bn = BatchNormalization()(out_3_1)

    
    conv_3_2 = Conv2D(256, kernel_size = 3, padding = 'same', activation = 'relu')(out_3_1_bn)
    conv_3_2 = Conv2D(256, kernel_size = 3, padding = 'same')(conv_3_2)
    conv_3_2_bn = BatchNormalization()(conv_3_2)
    
    # ShortCut connection
    add_3_2 = add([out_3_1, conv_3_2])
    out_3_2 = Activation('relu')(add_3_2)
    out_3_2_bn = BatchNormalization()(out_3_2)
    
    pool_3 = MaxPool2D((2,2), strides = 2)(out_3_2_bn)
    
    conv_4_0 = Conv2D(256, kernel_size = 1, strides = 1)(pool_3)
    conv_4_0_bn = BatchNormalization()(conv_4_0)
    
    conv_4_1 = Conv2D(256, kernel_size = 3, padding = 'same', activation = 'relu')(conv_4_0_bn)
    conv_4_1 = Conv2D(256, kernel_size = 3, padding = 'same')(conv_4_1)
    conv_4_1_bn = BatchNormalization()(conv_4_1)
    
    # ShortCut connection
    add_4_1 = add([conv_4_0_bn, conv_4_1_bn])
    out_4_1 = Activation('relu')(add_4_1)
    out_4_1_bn = BatchNormalization()(out_4_1)
    
    pool_4 = MaxPool2D((2,2), strides = 2)(out_4_1_bn)
    
    # FC layers
    img_features = Flatten()(pool_4)
    img_features = Dense(512, activation = 'relu')(img_features)
    img_features = Dropout(rate = 0.5)(img_features)
    img_features = Dense(512, activation = 'relu')(img_features)
    img_features = Dropout(rate = 0.5)(img_features)
    
    # Output layer
    digit_pred = Dense(10, activation = 'softmax')(img_features)
    
    model = Model(inputs = img_input, outputs = digit_pred)
    
    model.compile(optimizer = Adam(lr = learning_rate),
                 loss = 'categorical_crossentropy',
                 metrics = ['accuracy'])
                    
    return model

# Re-run part

In [ ]:
epochs = 50
batch_size = 100
num = 5
model_list = []
acc_list = []

for i in range(num):

    model = get_model(0.001)

    MODEL_SAVE_FOLDER_PATH = './drive/My Drive/DACON/saved_model/model_ResNet/'
    if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
        os.mkdir(MODEL_SAVE_FOLDER_PATH)

    model_path = MODEL_SAVE_FOLDER_PATH + '{}'.format(i) + '_{val_accuracy:.4f}.hdf5'

    # callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
    mcp_save = ModelCheckpoint(filepath = model_path, save_best_only=True, monitor='val_loss', mode='min', verbose=1)
    # reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_delta=1e-4, mode='min')

    # fit model
    x_train, x_val, y_train, y_val = train_test_gen(train_copy, 2)

    hist = model.fit(x_train, y_train, batch_size=batch_size, epochs = epochs, 
                validation_data = (x_val,y_val),
                steps_per_epoch=x_train.shape[0]// batch_size, 
                callbacks=[early_stopping,mcp_save])
    model = load_best('model_ResNet')

    model_list.append(model)
    acc_list.append(hist.history['val_accuracy'][-1])


In [ ]:
model_list

In [ ]:
x_test = np.divide(test_copy.iloc[:,2:].values,255)
x_test = x_test.reshape(-1,28,28,1)
pred = ensemble(x_test)
data = {'id':test_copy['id'], 'digit':pred}
submission = DataFrame(data)

file_name = 'submission_ensemble_ResNet_5'
submission.to_csv('./drive/My Drive/DACON/submission/'+ file_name +'.csv', index=False)

In [ ]:
pred[:10]

In [ ]:
file_list = [ 'submission_84',
             'submission_85',
             'submission_86_xgb_ensemble',
             'submission_87_ensembles',
             'submission_87_ensembles_10+5_bn_linearreg',
             'submission_88_ensemble_2_2_4_try3',
             'submission_88_ensembles_10+1_bn_linearreg',
             'submission_88_ensembles_6+2_bn_linearreg_2',
             'submission_89_ensemble_2_2',
             'submission_89_ensembles_stack_more',
             'submission_89_ensembles_stack_more_using_test',
             'submission_89_ensembles_stack_more++_using_test_overlap_wobn_512',
             'submission_90_ensembles_6+2_bn_08_retry',
             'submission_90_pretrain_using_test_layer_4_3ensemble',
             'submission_90_ensembles_6+2_bn_linearreg',
             'submission_90_ensembles_linear_using_test_1000',
             'submission_90_ensembles_stack_more+_using_test',
             'submission_91_ensembles_3+1_w1',
             'submission_91_ensembles_6+2_bn_08',
             'submission_91_ensembles_stack_more++_using_test_overlap_909191_2048_aug',
             'submission_91_ensembles_stack_more++_using_test_overlap_909191_64_aug'
             ]
pred_acc(file_name,file_list)